# 使用Pytorch做文本分类

文本分类是自然语言处理中最基本的任务之一，我们可以根据已有的文本分类结果进行有监督训练后进行分类，也可以根据一定的要求将文本进行聚类。我们今天做一个最简单的基于全连接层的词语情感分类。

In [ ]:
import torch
from matplotlib import pyplot as plt
from IPython import display
import numpy as np
import collections

我们首先来读取word2vec的矩阵，这个词向量矩阵是从Chinese-Word-Vectors提供的词向量矩阵中提取的，大小比较小，读取很快。

In [ ]:
char_list = []
emb_list = []

# 读取切分好的一行，返回词和词向量（numpy的矩阵）
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')

with open('素材\sgns.wiki.char', 'r', encoding='utf-8') as emb_file:
    # 文件的开头是词表长度和词嵌入维度
    dict_length, emb_size = emb_file.readline().rstrip().split()
    print('dict_length: ', dict_length)
    print('emb_size: ', emb_size)
    dict_length, emb_size = int(dict_length), int(emb_size)
    # 对每一行做处理，结果存到顺序词典中
    emb = collections.OrderedDict(get_coefs(*l.rstrip().split()) for l in emb_file.readlines())
for k, v in emb.items():
    print(k, v.shape)
    break

word2vec会将相似的词放在一起，我们来测试一下。我用的这个word2vec预训练词向量是基于词训练的，但我删掉了所有词只剩单字，所以单字上的找相似字的效果表现不是很好。一般用余弦相似度来确定两个向量之间的相似度。
$$\cos(\theta)=\frac{x\cdot y}{\parallel x\parallel\times\parallel y\parallel}$$


In [ ]:
def cal_cos_similarity(vec1, vec2):
    vec1 = np.mat(vec1)
    vec2 = np.mat(vec2)
    num = float(vec1 * vec2.T)
    denom = np.linalg.norm(vec1) * np.linalg.norm(vec2)
    # 防止除零错误
    eps = 1e-5
    sim = num / (denom + eps)
    return sim

def cal_char_similar(c):
    c_vec = emb.get(c)
    most_similar = ''
    max_cos = -1
    for k, v in emb.items():
        if k == c:
            continue
        tmp = cal_cos_similarity(v, c_vec)
        if tmp > max_cos:
            max_cos = tmp
            most_similar = k
    return most_similar

print(cal_char_similar('牛'))
print(cal_char_similar('鱼'))
print(cal_char_similar('难'))

我们知道，电脑无法直接处理文字，或者说，比较难以通过“文字”来找到对应的输入，因此我们在处理文本的时候，往往需要一个工具，将文本中的词或者字映射到一个数字ID，我们一般叫这个工具Tokenizer，这个没有太好的中文翻译，一般翻译为分词器，但是中文和英语不同，中文往往是多个字组成一个词，所以分词器应该是Segmenter。  
举个例子：
- Tokenizer： 翻/译/中/遇/到/的/难/点
- Segmenter:  翻译/中/遇到/的/难点

首先我们来生成一个Tokenizer类，很多时候Tokenizer也负责切分文本的作用。

In [ ]:
class Tokenizer:
    # 初始化的时候读取词表
    def __init__(self, vocab_list):
        self.vocab = self.load_vocab(vocab_list)
        for i, (k, v) in enumerate(self.vocab.items()):
            if i > 9:
                break
            print(k, v)
    
    # 读取词表
    def load_vocab(self, vocab_list):
        # 我们一般使用顺序字典来存储词表，这样能够保证历遍时index升序排列
        vocab = collections.OrderedDict()
        # 一般我们使用'UNK'来表示词表中不存在的词，放在0号index上
        vocab['UNK'] = 0
        index = 1
        # 依次插入词
        for token in vocab_list:
            token = token.strip()
            vocab[token] = index
            index += 1
        return vocab

    # 将单个字/词转换为数字id
    def token_to_id(self, token):
        # 不在词表里的词
        if token not in self.vocab.keys():
            return self.vocab['UNK']
        else:
            return self.vocab[token]

    # 将多个字/词转换为数字id
    def tokens_to_ids(self, tokens):
        ids_list = list(map(self.token_to_id, tokens))
        return ids_list

我们的词表就是之前的词向量里的“词”，所以我们可以现在来生成Tokenizer实例了。

In [ ]:
tokenizer = Tokenizer(emb.keys())

生成tokenizer实例的同时，我们也能够生成我们的词嵌入矩阵了。词嵌入矩阵就是第i行对应的就是编号为i的词的词向量。

In [ ]:
# 生成一个全0矩阵，大小为（词典长度+1，嵌入维度）
emb_matrix = np.zeros((1 + dict_length, emb_size), dtype='float32')

for word, id in tokenizer.vocab.items():
    emb_vector = emb.get(word)
    if emb_vector is not None:
        # 将编号为id的词的词向量放在id行上
        emb_matrix[id] = emb_vector
print(emb_matrix.shape)

接下来还是一样，我们来搭建NN。

In [ ]:
from torch import nn

class LinearClassifierNet(nn.Module):
    def __init__(self, seq_length, label_len):
        super(LinearClassifierNet, self).__init__()
        self.seq_length = seq_length
        self.label_len = label_len
        # 第一层是一个嵌入层，输入为(batch_size, seq_length),输出为(batch_size, seq_length, emb_size)
        # 嵌入层如果使用了from_pretrained，会关掉自动梯度，也就是变得不能训练。如果需要可以手动开启。
        self.emb = nn.Embedding.from_pretrained(torch.tensor(emb_matrix))
        self.emb_size = self.emb.embedding_dim
        # ReLU层无参数，可以共用
        self.relu = nn.ReLU()
        # 第一个全连接层，输入为(batch_size, seq_length*emb_size)，输出为(batch_size, 100)
        self.linear1 = nn.Linear(seq_length * emb_size, 100)
        # dropout层
        # Batch Norm和dropout一般不同时使用，如果你想同时用的话，顺序一般是：
        # CONV/FC -> BatchNorm -> ReLU/Sigmoid/tanh/GeLU(或者别的激活函数) -> Dropout -> ...
        # 参考论文 https://arxiv.org/abs/1502.03167
        self.dropout = nn.Dropout(p=0.3)
        # 第二个全连接层，输入为(batch_size, 100)，输出为(batch_size, 20)
        self.linear2 = nn.Linear(100, 20)
        # 第三个全连接层，输入为(batch_size, 20)，输出为(batch_size, label_len)
        self.linear3 = nn.Linear(20, self.label_len)
        # softmax分类层
        self.softmax = nn.Softmax(dim=-1)
        # 使用交叉熵损失函数
        # 交叉熵损失函数实际上等于nn.Softmax+nn.NLLLoss（负对数似然损失），所以用这个损失的时候不需要先过softmax层
        self.loss = nn.CrossEntropyLoss()

    # forward 定义前向传播，参数不同，输出结果也不同
    def forward(self, x, y=None):
        # 嵌入层，输出为(batch_size, seq_length, emb_size)
        x = self.emb(x)
        # 把输出的后两维拍平，变成一维
        x = x.view(-1, self.seq_length * self.emb_size)
        # 过第一个线性层
        x = self.linear1(x)
        # 非线性激活函数
        x = self.relu(x)
        # dropout层
        x = self.dropout(x)
        # 过第二个线性层
        x = self.linear2(x)
        # 非线性激活函数
        x = self.relu(x)
        # 过第三个线性层
        x = self.linear3(x)
        
        # 如果没有输入y，那么是在预测，我们返回分类的结果
        if y is None:
            return self.softmax(x)
        # 如果有输入y，那么是在训练，我们返回损失函数的值
        else:
            return self.loss(x, y)
        
# 我们做的是词语的情感分析，最长为5
seq_length = 5
# 情感只有正负两类
label_len = 2

model = LinearClassifierNet(seq_length, label_len)
# 使用print可以打印出网络的结构
print(model)

total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(str(total_trainable_params), 'parameters is trainable.')

if torch.cuda.is_available():
    model.to(torch.device('cuda'))

接下来我们来做数据的操作。一般来说我建议大家建立两个类，一个类放原始的数据和标签，一个类放处理完毕的数据和标签。

In [ ]:
# 原始数据和标签 
class data_example:
    def __init__(self, text, label):
        self.text = text
        self.label = label

# 处理完毕的数据和标签
class data_feature:
    def __init__(self, ids, label):
        self.ids = ids
        self.label = label

In [ ]:
# 读原始数据
examples = []
with open('素材/sentiment/正面情感词语（中文）.txt', 'r', encoding='gbk') as pos_file:
    for line in pos_file:
        line = line.strip()
        examples.append(data_example(line, 'positive'))
with open('素材/sentiment/负面情感词语（中文）.txt', 'r', encoding='gbk') as pos_file:
    for line in pos_file:
        line = line.strip()
        examples.append(data_example(line, 'negative'))

print('num of example: %d' % len(examples))
for i in range(3):
    print(examples[i].text, examples[i].label)

然后我们将原始数据转换成处理完毕的数据和标签。这么做主要是为了解耦。

In [ ]:
# 处理原始数据
def convert_example_to_feature(examples):
    features = []
    for i in examples:
        # 使用tokenizer将字符串转换为数字id
        ids = tokenizer.tokens_to_ids(i.text)
        # 我们规定了最大长度，超过了就切断，不足就补齐（一般补unk，也就是这里的[0]，也有特殊补位符[PAD]之类的）
        if len(ids) > seq_length:
            ids = ids[0: seq_length]
        else:
            ids = ids + [0] * (seq_length - len(ids))
        # 如果这个字符串全都不能识别，那就放弃掉
        if sum(ids) == 0:
            continue
        assert len(ids) == seq_length
        # 处理标签，正面为1，负面为0
        if i.label == 'positive':
            label = 1
        else:
            label = 0
        features.append(data_feature(ids, label))
    return features

features = convert_example_to_feature(examples)

for i in range(3):
    print(features[i].ids, features[i].label)

然后还是熟悉的Dataset和DataLoader，将数据生成Dataset然后用DataLoader去读取~

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

ids = torch.tensor([f.ids for f in features], dtype=torch.long)
label = torch.tensor([f.label for f in features], dtype=torch.long)

dataset = TensorDataset(ids, label)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

因为网络比较复杂，我们使用Adam优化器，这个优化器和它的改良版AdamW可以说是目前用的最多最广的优化器之一。

In [ ]:
from torch.optim import Adam

optimizer = Adam(model.parameters(), lr=0.001)
print(optimizer)

然后就是开始训练啦！依然是熟悉的套路：

In [ ]:
epoch = 9
for i in range(epoch):
    total_loss = []
    for ids, label in dataloader:
        if torch.cuda.is_available():
            ids = ids.to(torch.device('cuda'))
            label = label.to(torch.device('cuda'))
        # 因为我们这次loss已经写在模型里面了，所以就不用再计算模型了
        optimizer.zero_grad()
        loss = model(ids, label)
        total_loss.append(loss.item())
        loss.backward()
        optimizer.step()
    print("epoch: %d, loss: %.6f" % (i + 1, sum(total_loss) / len(total_loss)))

我们可以来检验一下模型的准确性，这时我们之前写forward时用到的“不同参数不同输出”就有用了，在不传入标签的时候，会直接做预测。

In [ ]:
# 将输出的概率还原成标签
def tensor_to_label(logits):
    logits = logits.detach().cpu().numpy()
    logits = np.argmax(logits, axis=-1)
    if logits[0] == 1:
        return 'positive'
    else:
        return 'negative'

# 还记得网络中我们加了dropout吗？
# 当我们将model设置为eval状态时，dropout/BatchNorm不生效
model.eval()

while True:
    s = input()
    if s == 'quit':
        break
    s = [data_example(s, 0)]
    s = convert_example_to_feature(s)
    ids = torch.tensor([f.ids for f in s], dtype=torch.long)
    with torch.no_grad():
        if torch.cuda.is_available():
            ids = ids.to(torch.device('cuda'))
        logits = tensor_to_label(model(ids))
        print(logits)

这节课开始我们正式开始接触各种NLP任务，首先是最简单的全连接做文本分类，大家可以看到5个字的文本分类就已经需要15w+的参数了，因此全连接层在NLP任务的前几层是很少见的，一般会用在最后面几层。之后我们会再给大家讲讲CNN方法做文本分类。  
此外，这节课的代码其实也是一般NLP任务的常用代码框架，希望大家能够读懂代码结构，以后能够很大程度少走弯路。